**Course organisers**

Jan Grohn (jan.grohn@psy.ox.ac.uk), Miriam Klein-Flügge (miriam.klein-flugge@psy.ox.ac.uk)  


# Introduction and recap
**Aims for today’s session:**
- Fit the computational model to some real data
- Compare fitted models

In this session we will finally analyse some data from real participants. The dataset we have chosen is from a paper by [Blain and Rutledge](https://elifesciences.org/articles/57977) and is available [here](https://archive.softwareheritage.org/browse/revision/b7c4a0cd761dcf249c72caf809dd81af24c4a49b/). You don't have to download the dataset yourself, our code will do that for you. For now, read through the paper to see what design choices Blain and Rutledge made to run the task. How is the task they ran similar or different to what we have been discussing so far? Hint: to answer this question you don't have to read the entire paper from beginning to end, it is sufficient to skim-read the paper to identify the relevant sections in which the authors describe how they ran their task.

→ Type your answer here

In [ ]:
#@title ## Import libraries

# numpy is a libarary used to do all kinds of mathematical operations
import numpy as np

# pandas allows us to organise data as tables (called "dataframes")
import pandas as pd

# we are using the chi2 distribution for some statistical tests
from scipy.stats import chi2

# this function allows us to perform one sample t-tests
from scipy.stats import ttest_1samp

# this allows us to make interactive figures
from google.colab import output
output.enable_custom_widget_manager()

# seed the random number genrator
rng = np.random.default_rng(12345)

# download the dataset from Blain & Rudledge 2020.
!wget "https://github.com/BastienBlain/MSWB_LearningNotReward/raw/main/PublicCode/Blain_MoodTracksLearning_data.mat"

# load in some custom functions for this block practical
!rm -rf ComputationalModelingBlockPractical
!git clone https://github.com/jangrohn/ComputationalModelingBlockPractical
!cp ComputationalModelingBlockPractical/session3/*py .
!rm -rf ComputationalModelingBlockPractical
!rm -rf sample_data
from plotting import *
from fitting import *



The dataset contains data from 75 participants playing the task. To visualise the data from the first participant (with ID 0), run the next code cell. The way we are plotting participant choices now is slightly different from how we did it throughout the last two sessions: now the position of the choice marker (0 or 1) tells you which option the participant chose, while the colour (green or red) tells you whether they received a reward or not.

In [ ]:
plot_schedule(0)

The last participant's data (ID 74) can be visualised in a similar way:

In [ ]:
plot_schedule(74)

## Recap – Reinforcement learning model
During the last two weeks, we have motivated and built a computational model of learning and decision making. You should be fairly familiar with these equations by now, but we will quickly recap them here. On each trial $t$, the model computes a prediction error, which is the difference between the observed and the predicted outcome:

$$
\underbrace{\delta_t}_\textrm{prediction error} = \underbrace{o_t}_\textrm{outcome} - \underbrace{p_t}_\textrm{model prediction} \tag{Equation 1}
$$

The model then uses this prediction error to make a new prediction for the next trial. This is done by updating the prediction proportional to the prediction error, scaled by a constant $\alpha$, which we call the learning rate.

$$
\underbrace{p_{t+1}}_\textrm{new prediction} = \underbrace{p_t}_\textrm{old prediction} + \underbrace{\alpha \delta_t}_\textrm{scaled prediction error} \tag{Equation 2}
$$

During the task we are modelling, participants had to take two variables into account when making their choices: the probability that an option is rewarded, which we are modelling according to the above equations, and a number of reward points that are on offer. To make choices, they have to somehow integrate these two variables. During last week's session (Session 2), we discussed two ways of integrating reward probability and magnitude. Multiplicative utility assumes that participants multiply the magnitude and probabilty

$$
\underbrace{u}_\textrm{utility} = \underbrace{m}_\textrm{reward magnitude} \times \underbrace{p}_\textrm{reward probabilty} \tag{Equation 3}
$$

whereas additive utility assumes that participants compute a weighted average of magnitude and probabilty:

$$
\underbrace{u}_\textrm{utility} = \overbrace{\omega}^\textrm{magnitude weight} \times \underbrace{m}_\textrm{reward magnitude} + \overbrace{(1-\omega)}^\textrm{probability weight} \times \underbrace{p}_\textrm{reward probabilty} \tag{Equation 4}
$$

Finally, we need to translate the utility we computed into actual choices. To do so, we assume that participants pick each option with a certain probability, which depends on the difference in utility between the two options and a 'randomness factor', which we call the inverse temperature $\beta$:
$$
\underbrace{P(c_1)}_\textrm{probability of choosing option 1} = \frac{1}{1+ e^{-\beta(u_1 - u_2)}} \tag{Equation 5}
$$

## Recap – Model fitting

Next, we learned how to fit models by maximising the model likelihood. A good model fit (i.e., one with a high likelihood) assigns a high softmax probability (Equation 5) to the actual (or simulated) choices a participant made. We want to find the parameters of the model $\alpha$, $\beta$, and $\omega$ (if we use additive utility) that give us the highest likelihood, as these are, according to the model, the most plausible parameters this participant might have used to solve the task.


# Section 1: Fitting and comparing models

## Multiplicative utility
We will now see whether the predictions we motivated throughout the last two sessions are indeed what Blain and Rutledge found. In particular, we predicted that participants should have a higher learning rate in a volatile block than in a stable block. The first step towards testing whether this prediction is correct is by fitting learning rates to the data from all participants. This is done in the next code cell. For now, we are assuming multiplicative utility.  

In [ ]:
data1AlphaMul, data2AlphaMul = fit_participant_data(multiplicative_utility)

The function returns two tables, one containing learning rates that were fitted assuming a model where the same learning rate is used in the stable and the volatile block, and one assuming that different learning rates were used. As a first visual approximation to check our hypothesis, we can plot the distributions of the learning rates in the stable and the volatile block.

### Comparing fitted learning rates

In [ ]:
visualise_alpha_distributions(data2AlphaMul.alphaStable, data2AlphaMul.alphaVolatile, 'fitted learning rates assuming multiplicative utility')

Based on visually inspecting these distributions, do you think the learning rates are higher in the volatile compared to the stable session? Do you think a plot like this allows you to make this kind of judgement?

→ Type your answer here

One problem with the above way of visualising the distributions is that it is not particularly well suited for answering our key question. A plot like the one above is useful in determening whether the mean of the distribtuion of learning rates fitted to the volatile block is larger than the mean of the disribution of learning rates fitted to the stable block. An example of a statistical test that would allow you to compute a p-value for such a comparison is an unpaired t-test.

However, in our case the question we want to ask is slightly different. We want to ask whether the learning rate in the volatile session is larger than in the stable session *for every participant*. In other words, we need to use a paired t-test, where we compare the *difference* in learning rates on a participant-by-participant basis. Thus, a much more useful visualization would be the difference in learning rates between the two session types (computed for each participant):

In [ ]:
visualise_alpha_difference(data2AlphaMul.alphaStable, data2AlphaMul.alphaVolatile, 'learning rates fitted to participant data assuming multiplicative utility')

If this data looks approximately normal (i.e., its distribution looks like a normal distribution), then we can perform a t-test to see whether this difference is larger than 0. This is done in the next code cell:

In [ ]:
ttest_1samp(data2AlphaMul.alphaVolatile - data2AlphaMul.alphaStable, 0, alternative = "greater")

How do you interpret this result? Do you think everything we have done in our analysis up until this point is valid?

→ Type your answer here

### Model comparisons

While we found a significant result with our t-test, we can (and perhaps should) take a step back and ask ourselves whether the model we used to fit the learning rates was correct in the first place. Were we justified in fitting separate learning rates? Were we correct in assuming multiplicative utility, or should we have used some other utility function? Let us tackle the fist of these two questions now.

We have now fitted the model to each participant twice, once assuming that $\alpha$ has the same value in the volatile and the stable session, and once allowing it to take on a different value in each session. We now need to determine which of these model fits describes the participants' data better. This is a model comparison problem, where we have two competing models. To decide which test to use to perform this model comparison, we should first consider whether the two model fits we want to compare are nested or not. Models are nested if the more complex model (the one with more parameters) can be transformed into the simpler model by imposing constraints on the parameters. Do you think the two models we fitted to the data are nested or not?


→ Type your answer here


For nested models only, we can test whether the more complex model (called the 'alternative model') describes the data better than the simpler model (called the 'null model') by performing a likelihood ratio test. The likelihood ratio test statistic can be computed as shown in this equation:

$$
\underbrace{\lambda_{\textrm{LR}}}_\textrm{likelihood ratio test statistic} = 2 \log \left( \frac{\textrm{likelihood for the alternative model}}{\textrm{likelihood for the null model}} \right)  \tag{Equation 6}
$$

In our case, as described in the last session, we were fitting the models by finding the maximum of their log likelihood distributions. Another way of writing Equation 6 is by noticing that taking the log of a ratio is equal to subtracting the logs of the numerator and denominator.

$$
\lambda_{\textrm{LR}} = 2 \left(\log(\textrm{likelihood for the alternative model}) - \log (\textrm{likelihood for the null model})\right)  \tag{Equation 7}
$$

Thus, Equation 7 is a handy way to express the likelihood ratio test statistic as it allows us to directly substitute our fitted log likelihoods into the equation. This is done in the next cell. Because we are dealing with log likelihoods, we can also just simply sum up the likelihoods of each individual participants' model fit to obtain the overall log likelihood of that model.




In [ ]:
lambda_LR = 2*sum(data2AlphaMul.LL - data1AlphaMul.LL)


This test statistic is neccessarily always positive because the log likelihood for the alternative model must be equal or larger than the log likelihood for the null model. Can you explain why this is true?  

→ Type your answer here

To get a p-value for this test statistic, we need to know what distribution it follows. According to [Wilks' theorem](https://en.wikipedia.org/wiki/Wilks%27_theorem), as the sample size approaches infinity, $\lambda_{\textrm{LR}}$ asymptotially approaches the $\chi^2$ distribution under the null hypothesis, with degrees of freedom equal to the difference in parameters between the alternative and the null model. What do you think is this difference in parameters in our case? You also need to fill in the degrees of freedom in the next code cell that computes the p-value according to the $\chi^2$ value of the likelihood ratio test statistic.

→ Type your answer here

In [ ]:
degrees_of_freedom = # enter the degrees of freedom here
p_value = chi2.sf(lambda_LR, degrees_of_freedom)
print('Chi2(' + str(degrees_of_freedom) + ') = ' + str(lambda_LR) + ', p = ' + str(p_value))

To understand this p-value and the likelihood ratio test a bit better, we can generate some simulated data where we know the ground truth, just like we did in the last session. First, let us simulate a dataset in which there is no difference between the learning rates in stable and volatile sessions and fit it just like we fitted the real data. This time, the dataset we simulate will use the exact same schedule that participants saw during the experiment. For simplicity, we will use the fitted learning rates in the stable session to now simulate behaviour both in the stable and the volatile session. We will also use the fitted inverse temperatures. Do you think this approach is sensible or could there be any potential problems with it? Because the simulation and fit might take a while, you can already run the next code cell while typing your answer.

→ Type your answer here

In [ ]:
data1AlphaMulSimSame, data2AlphaMulSimSame = fit_participant_data(
    multiplicative_utility,
    simulate=True,
    alpha_S =
    data2AlphaMul.alphaStable,
    alpha_V = data2AlphaMul.alphaStable,
    beta = data2AlphaMul.beta,
    rng = rng)


The next code cell visualises the recovered learning rates in the stable and the volatile sessions.

In [ ]:
visualise_alpha_distributions(data2AlphaMulSimSame.alphaStable, data2AlphaMulSimSame.alphaVolatile, 'recovered learning rates assuming multiplicative utility and no ground truth difference')
visualise_alpha_difference(data2AlphaMulSimSame.alphaStable, data2AlphaMulSimSame.alphaVolatile, '')

We can now perform a likelihood ratio test on the recoverd learning rates to determine whether we can reject the (true) null hypothesis that they are equal in the stable and volatile session. Before running the next code cell, predict whether you expect this p-value to be signficant or not, and why.

→ Type your answer here

In [ ]:
lambda_LR = 2*sum(data2AlphaMulSimSame.LL - data1AlphaMulSimSame.LL)
p_value = chi2.sf(lambda_LR, degrees_of_freedom)
print('Chi2(' + str(degrees_of_freedom) + ') = ' + str(lambda_LR) + ', p = ' + str(p_value))

To contextualise this result and the likelihood ratio test even further, we will now run one more simulation. Now we will be using the same learning rates in the stable and the volatile sessions, but we will randomly shuffle the order of the learning rates in the volatile session. That is, we will use the same distribution of learning rates but which learning rate is used to simualte data for each participant will be different in the volatile sessions.

In [ ]:
data1AlphaMulSimShuffled, data2AlphaMulSimShuffled = fit_participant_data(
    multiplicative_utility,
    simulate=True,
    alpha_S = data2AlphaMul.alphaStable,
    alpha_V = rng.permutation(data2AlphaMul.alphaStable),
    beta = data2AlphaMul.beta,
    rng = rng)

The next cell plots the recovered distributions of learning rates.

In [ ]:
visualise_alpha_distributions(data2AlphaMulSimShuffled.alphaStable, data2AlphaMulSimShuffled.alphaVolatile, 'recovered learning rates assuming multiplicative utility and shuffled ground truth')
visualise_alpha_difference(data2AlphaMulSimShuffled.alphaStable, data2AlphaMulSimShuffled.alphaVolatile, '')

We can now again perform a likelihood ratio test. Before running the next code cell, predict whether the p-value you obtain will be significant. That is, for this simulated dataset, can we reject the null hypothesis that the learning rates in the stable and the volatile sessions are equal?

→ Type your answer here

In [ ]:
lambda_LR = 2*sum(data2AlphaMulSimShuffled.LL - data1AlphaMulSimShuffled.LL)
p_value = chi2.sf(lambda_LR, degrees_of_freedom)
print('Chi2(' + str(degrees_of_freedom) + ') = ' + str(lambda_LR) + ', p = ' + str(p_value))

## Additive Utility

We have now determined whether a model assuming two learning rates is a better fit to the data than a model assuming one learning rate. However, all the above analyses assumed that participants employ multiplicative utility. Let us now explore whether some other utility function provides a better fit. To do so, the next code cell will fit the participant data using additive utility. Run the next code cell, which might take a while, and continue with the next section while the model is being fit to the data.

In [ ]:
data1AlphaAdd, data2AlphaAdd = fit_participant_data(additive_utility, method = 'Nelder-Mead')

### Model comparisons

To do a model comparison between a model using multiplicative utility and a model using additive utility, we again first have to determine whether the two models are nested or not. Why do you think they are nested or not nested?

→ Type your answer here

For nested models, we established earlier that the likelihood for the model with more parameters is always equal or greater than the likelihood of the model with less parameters. Is this also true for non-nested models (i.e., that the likelihood of the model with more parameters is equal or greater than the likelihood of the model with less parameters)?

→ Type your answer here

For models that are not nested, we are not allowed to do a likelihood ratio test. An alternative to a likelihood ratio is to compute the Bayesian information criterion (BIC) for each model and compare these. Unlike for likelihood ratio tests, we cannot compute a p-value when comparing models using BICs. As such, the BIC should be regarded as more of a heuristic for model comparison rather than a statistal test. BICs can be computed using the following equation:

$$ \textrm{BIC} = k \log (n)- 2 \log (\textrm{likelihood}) \tag{Equation 3}$$

where $k$ is the number of free parameters of the model and $n$ is the number of data points (or observations, trials, etc.). A model with a lower BIC is considered the better model. This is also reflected in the formula: consider two models A and B fitted to the same dataset that have the same likelihood, but model A has more free parameters $k$ than model B. In this case, model B will have a lower BIC as it manages to achieve the same goodness of fit (i.e. likelihood) with less degrees of freedom.

Once the model fit with additive utility has finished running above, you can plot the fitted learning rates using the code below:

In [ ]:
visualise_alpha_distributions(data2AlphaAdd.alphaStable, data2AlphaAdd.alphaVolatile, 'fitted learning rates assuming additive utility')
visualise_alpha_difference(data2AlphaAdd.alphaStable, data2AlphaAdd.alphaVolatile, '')

To compare the BICs of the two models, run the next code cell. What does the BIC comparison suggest?

In [ ]:
print('BIC assuming additive utility and two learning rates: ' + str(sum(data2AlphaAdd.BIC)))
print('BIC assuming multiplicative utility and two learning rates: ' + str(sum(data2AlphaMul.BIC)))

→ Type your answer here

Just like we did before, we can also perform a t-test to check if the learning rates are larger in the volatile session:

In [ ]:
ttest_1samp(data2AlphaMul.alphaVolatile - data2AlphaMul.alphaStable, 0, alternative = "greater")

And we can do a likelihood ratio test to check if a model with two learning rates outperforms a model with one learning rate (now assuming additive utility for both):

In [ ]:
lambda_LR = 2*sum(data2AlphaAdd.LL - data1AlphaAdd.LL)
p_value = chi2.sf(lambda_LR, degrees_of_freedom)
print('Chi2(' + str(degrees_of_freedom) + ') = ' + str(lambda_LR) + ', p = ' + str(p_value))

Given all the analyses above, what can you conclude about the data so far?

→ Type your answer here

# Section 2: Excluding participants

What we really should have done before jumping head first into fitting models to the data is to carefully consider whether any participants were not doing the task correctly, and excluding them from the analysis. What we mean by doing the task correctly can take many forms, such as taking too many or too long breaks while doing the task, or not understanding the instructions (which entails doing the task in a 'strange' way).

To see whether participants are disengaging from the task, we can consider taking a look at their response times. Analysing response time data would also allow us to see if participants respond too fast, which might indicate that they are making random responses without considering the options on the screen. However, in the interest of time, we will not look at response times here. Instead, we will limit ourselves to considering the choices participants made to decide whether to include or exclude them.

To this end, it makes a lot of sense to fit models to the choice data first, and to use these model fits to diagnose whether to exclude participants. Remember, the question we want to answer is whether participants have a higher learning rate in the volatile compared to the stable block. One way to interpret this question is that we expect participants *that learn about the probabilities of the options* to have a higher learning rate. Under this interpretation, the learning rates we fit to participants that do not learn about probabilities but solve the task in some other way are not meaningful, and there might thus be good reason to exclude them from our analysis.

Using the model fits to diagnose whether to exclude participants can be done by reminding ourselves of the meaning of the model parameters. For example, for the learning rate $\alpha$, what does it imply if a learning rate is at its lower extreme (0) or higher extreme (1)? Can we still claim that participants *learned* in these extreme cases? Does this depend on the utility function we used? Similarly, for the inverse temperature $\beta$, what does it mean if it is at its lower extreme (0) or higher extreme (approaching infinity)? Finally, what do extreme values of $\omega$ (0 or 1) imply for the strategies participants use? Having reminded yourself of the meaning of these parameters, for which extreme values of which parameters would you consider excluding participants, and why?

→ Type your answer here

## Visualising parameter correlations

Let us now plot the distributions of fitted parameters for both utility functions. First for multiplicative utility:

In [ ]:
plot_parameter_corrs(data2AlphaMul)


And also for additive utility:

In [ ]:
plot_parameter_corrs(data2AlphaAdd)


Is there anything in these plots that stands out? You can hover over the dots in the plots to see the participant ID associated with a datapoint. You can then use this ID to examine the data for a particular participant of interest further. For example, to examine the data of the first participant (with ID 0), we can look at the fitted values of the additive utility model like this:

In [ ]:
data2AlphaAdd[data2AlphaAdd.ID == 0]

Similarly, we can examine the fitted parameters for this participant when fitted with multipliative utility like this:

In [ ]:
data2AlphaMul[data2AlphaMul.ID == 0]

## Visualising model-internal variables

We can also visualise what the model thinks are the most likely trial-by-trial variables that produced the participant's choices:

In [ ]:
plot_schedule(0, data2AlphaAdd)

We can also check what a different model thinks happened during the task for this participant by passing the data frame associated with a different model fit to the function:

In [ ]:
plot_schedule(0, data1AlphaMul)

## Actually excluding participants

If we want to replot the parameters but exclude some participants, we can use code such as the following, which excludes the participants with IDs 0, 1, and 2:

In [ ]:
exclude = [0,1,2]
plot_parameter_corrs(data2AlphaAdd[~data2AlphaAdd.index.isin(exclude)])

Are there any participants you think should be excluded? If so, what are their IDs, and why do you think they should be excluded? What do they seem to be doing during the task?

→ Type your answer here

If you have decided to exclude some participants, you can now redo the statistical analyses we have completed above without taking these participants into account. For example, let's say you exclude participants 0, 1, and 2. The likelihood ratio test assuming additive utility now becomes:

In [ ]:
exclude = [0,1,2]
degrees_of_freedom = 72
lambda_LR = 2*sum(data2AlphaAdd[~data2AlphaAdd.index.isin(exclude)].LL - data1AlphaAdd[~data1AlphaAdd.index.isin(exclude)].LL)
p_value = chi2.sf(lambda_LR, degrees_of_freedom)
print('Chi2(' + str(degrees_of_freedom) + ') = ' + str(lambda_LR) + ', p = ' + str(p_value))

And the BIC comparison becomes:

In [ ]:
print('BIC assuming additive utility and two learning rates: ' + str(sum(data2AlphaAdd[~data2AlphaAdd.index.isin(exclude)].BIC)))
print('BIC assuming multiplicative utility and two learning rates: ' + str(sum(data2AlphaMul[~data2AlphaMul.index.isin(exclude)].BIC)))

Adapt all statistical tests and model comparisons throughout this notebook to exclude the participants you would like to exclude, and replot the figures. Do any of your conclusions change now that you have excluded participants?

→ Type your answer here

Alternatively, after examining individual participants, you might conclude that they are making sensible choices that are just not well described by any of the models we have fitted to the data. If you would like, you can now adapt the functions we used to fit the participant data to include variants of the models that might describe some participants even better. For example, here are some models that you could sensibly implement and test:
 - does having separate inverse temperatures for the stable and the volatile sessions describe some participants better?
 - should the initial value (i.e., the starting probability of the RL model) be fitted as a free parameter?
 - are there other utility functions you think participants might be using?